<a href="https://colab.research.google.com/github/roman-dubrovin/hackatons/blob/main/hack_lct24_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 970.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import os
import pandas as pd
import json
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# загрузка библиотеки spacy и русского словаря
import spacy
nlp = spacy.load("ru_core_news_sm")

import ru_core_news_sm
nlp = ru_core_news_sm.load()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# указываем путь к рабочей папке
path = '/content/drive/MyDrive/hack_lct24'

# указываем имя файла с тестовым датасетом
test_filename = 'gt_test.csv'

In [ ]:
# вспомогательная функция для определение номера токена в выборке по номеру его позиции
def find_pos(parser_array, n):
    pos = -1
    if n != -1:
        i = 0
        while i < len(parser_array):
            if parser_array[i]['pos'] == n:
                pos = i
                break
            i += 1
    return pos

# вспомогательная функция для корректировки отношения токена
def correct_relation(parser_array, n):
    i = find_pos(parser_array, n)
    if i != -1:
        if parser_array[i]['code'] == 'value':
            if parser_array[i]['relation'] != parser_array[i]['pos']:
                rel = correct_relation(parser_array, parser_array[i]['relation'])
                parser_array[i]['relation'] = rel
            else:
                rel = parser_array[i]['pos']
        else:
            rel = parser_array[i]['pos']
    else:
        rel = n

    return rel


# функция для поиска в заданном тексте интересующих слов
# ("скидка", числа, относящиеся к скидке, а также единицу измерения скидки: "процент" или "рубль")
def parser_text(inp_text):
    '''
    вход:
    inp_text - строка с входным текстом, в котором нуждной найти заданные слова

    выход:
    out_array - массив с маркерами, размерность которого равна кол-ву слов во входном тексте

    '''

    # обработаем входной текст с помощью spacy
    doc = nlp(inp_text)

    # запишем в inp_text_parser интересующие нас токены
    inp_text_parser = []
    for i in range(len(doc)):
        if doc[i].lemma_ in ['скидка', 'взнос', 'стоимость', 'цена', 'оплата', 'предоплата', 'ставка', 'ипотека', 'кредит']:
            # анализируемые сущности
            inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'entity'}]
        elif doc[i].lemma_ in ['процент', 'рубль', 'метр', 'квадрат', 'год', 'квартал', 'неделя', 'месяц', 'день', 'условие', 'этаж', 'корпус']:
            # измерения для чисел
            inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'measure', 'relation': -1}]
        elif doc[i].like_num:
            # числа
            inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'value', 'relation': doc[i].head.i}]

    # корректируем для чисел их отношение к измерению
    for i in range(len(inp_text_parser)):
        if inp_text_parser[i]['code'] == 'value':
            _ = correct_relation(inp_text_parser, inp_text_parser[i]['pos'])

    # удалим токены-числа и токены-измерения, если они относятся не к измерениям "процент" или "рубль"
    length_parser = len(inp_text_parser)
    i = 0
    while i < length_parser:
        if inp_text_parser[i]['code'] == 'value':
            j = find_pos(inp_text_parser, inp_text_parser[i]['relation'])
            if (j == -1) or not(inp_text_parser[j]['lemma'] in ['процент', 'рубль']):
                _ = inp_text_parser.pop(i)
                length_parser -= 1
            else:
                i += 1
        elif (inp_text_parser[i]['code'] == 'measure') and not(inp_text_parser[i]['lemma'] in ['процент', 'рубль']):
            _ = inp_text_parser.pop(i)
            length_parser -= 1
        else:
            i += 1

    # расчитаем расстояния между сущностями и началом блоков чисел и измерения слева и справа
    # если между сущностями отсутствуют числа и измерения, то установим -1
    length_parser = len(inp_text_parser)
    for i in range(length_parser):
        if inp_text_parser[i]['code'] == 'entity':
            left = -1
            if (i-1 >= 0) and (inp_text_parser[i-1]['code'] in ['value', 'measure']):
                left = abs(inp_text_parser[i]['pos'] - inp_text_parser[i-1]['pos'])
            inp_text_parser[i]['left'] = left
            right = -1
            if (i+1 < length_parser) and (inp_text_parser[i+1]['code'] in ['value', 'measure']):
                right = abs(inp_text_parser[i]['pos'] - inp_text_parser[i+1]['pos'])
            inp_text_parser[i]['right'] = right

    # удалим сущности не "скидка", у которых растояния одновременно слева и справа -1
    length_parser = len(inp_text_parser)
    i = 0
    while i < length_parser:
        if (inp_text_parser[i]['code'] == 'entity') and not(inp_text_parser[i]['lemma'] == 'скидка') and\
           ((inp_text_parser[i]['left'] == -1) and (inp_text_parser[i]['right'] == -1)):
            _ = inp_text_parser.pop(i)
            length_parser -= 1
        else:
            i += 1

    # установим для чисел и измерений их отношение к сущностям
    length_parser = len(inp_text_parser)
    for i in range(length_parser):
        if inp_text_parser[i]['code'] == 'entity':
            if (inp_text_parser[i]['left'] == -1) or ((inp_text_parser[i]['left'] >= inp_text_parser[i]['right']) and (inp_text_parser[i]['right'] != -1)):
                j = i+1
                while (j < length_parser) and (inp_text_parser[j]['code'] != 'entity'):
                    j += 1
                if i != j-1:
                    if (j == length_parser) or ((inp_text_parser[j]['code'] == 'entity') and (inp_text_parser[i]['right'] < inp_text_parser[j]['left'])):
                        pos = inp_text_parser[i]['pos']
                        for k in range(i+1, j):
                            inp_text_parser[k]['relation'] = pos
                            inp_text_parser[k]['marker'] = 'I-value'
                        inp_text_parser[i+1]['marker'] = 'B-value'
            elif (inp_text_parser[i]['right'] == -1) or ((inp_text_parser[i]['left'] < inp_text_parser[i]['right']) and (inp_text_parser[i]['left'] != -1)):
                j = i-1
                while (j > 0) and (inp_text_parser[j]['code'] != 'entity'):
                    j -= 1
                if (i != j+1):
                    if (j == 0) or ((inp_text_parser[j]['code'] == 'entity') and (inp_text_parser[i]['left'] < inp_text_parser[j]['right'])):
                        pos = inp_text_parser[i]['pos']
                        for k in range(j, i):
                            inp_text_parser[k]['relation'] = pos
                            inp_text_parser[k]['marker'] = 'I-value'
                        inp_text_parser[j]['marker'] = 'B-value'

    # удалим токены-числа и токены-измерения, если они относятся не к сущности "скидка"
    length_parser = len(inp_text_parser)
    i = 0
    while i < length_parser:
        if inp_text_parser[i]['code'] in ['value', 'measure']:
            j = find_pos(inp_text_parser, inp_text_parser[i]['relation'])
            if (j == -1) or not(inp_text_parser[j]['lemma'] in ['скидка']):
                _ = inp_text_parser.pop(i)
                length_parser -= 1
            else:
                i += 1
        elif inp_text_parser[i]['code'] == 'entity':
            if inp_text_parser[i]['lemma'] in ['скидка']:
                inp_text_parser[i]['marker'] = 'B-discount'
                i += 1
            else:
                _ = inp_text_parser.pop(i)
                length_parser -= 1
        else:
            i += 1

    # сформируем выходной массив
    out_array = ['O' for i in range(len(doc))]

    # расставим в выходном массиве маркеры для найденных слов
    for i in range(length_parser):
        pos = inp_text_parser[i]['pos']
        out_array[pos] = inp_text_parser[i]['marker']

    return out_array

In [ ]:
# загружаем тестовый датасет по указанному пути
df_test = pd.read_csv(os.path.join(path, test_filename))

In [ ]:
# выделим тестовые фразы в отдельный массив
test_texts = df_test['processed_text'].tolist()

In [ ]:
# выполним parser_text для каждой тестовой фразы
# сохраняя результат обработки в выходной массив
test_arr = []
for test_txt in tqdm(test_texts):
    test_arr += [parser_text(test_txt)]

100%|██████████| 482/482 [01:01<00:00,  7.80it/s]


In [ ]:
# обновим поле с метками в тестовом датасете
df_test['label'] = test_arr

In [ ]:
# сохраним датасет в submission.csv по указанному пути
df_test.to_csv(os.path.join(path, 'submission.csv'), index=False)

In [ ]:
df_train = pd.read_csv(os.path.join(path, 'train_data.csv'))

In [ ]:
df_train.head(20)

,processed_text,target_labels_positions
0,аа союзная тридцать пять дробь один лариса сое...,{}
1,аа приложение мне показывает к оплате у меня п...,{}
2,да лисное по призрак лишнее ну почему иду пять...,"{'I-value': [140], 'B-value': [139], 'B-discou..."
3,а что добрый день NAME у меня пришел какой то ...,{'B-discount': [12]}
4,у меня западный с утра да да еще да да самый в...,{}
5,NAME ну а по поводу ипотеки по моему ну москов...,{}
6,да пока здесь минутку я уразил жду врача да ин...,{}
7,да спа спал так подобное десять пар не сюда ну...,{}
8,а хата амально позвонили NAME да если можно ес...,{}
9,NAME добрый день NAME зовут а NAME сейчас назе...,"{'I-value': [117], 'B-value': [116], 'B-discou..."


In [ ]:
inp_text = df_train.iloc[2]['processed_text']
inp_text

'да лисное по призрак лишнее ну почему иду пять двух полностью последнего да да да и там есть данные надо чтобы стандартная двух что была пятьдесят пятьдесят десять миллионов три а первоначальный вздох сколько вас а еще какие то фиджи приедем там сейчас менеджера да вам я такая там проценты а если не столько да хорошо конечно и это заработал хорошо а инвестое отправьте пожалуйстаздравствуйте меня зовут на менеджере дело продаж группы самолет вы рассматриваете какой то определенный комплекс для себя что хотели просмотреть сколько комнат двух комнатная квартира так давайте посмотрим я вижу вы интересовались ранее да этим знаете где он находится о там есть да уже готовы какая площадь вас интересует секунду так ну вот готовая двухкомнатная пятьдесят квадратных метров с отделкой десять миллионов триста да минимум пятнадцать процентов миллион шестьсот продаж дополнительно могу вам отправить скидку два процента она действует в течение двух дней сегодня и завтра удобно сегодня к нам подъехать 

In [ ]:
#parser_text(inp_text)

In [ ]:
inp_text = 'скидку конечно я могу вам предложить например три процента вот такая скидка устроит'

doc = nlp(inp_text)

# запишем в inp_text_parser интересующие нас токены
inp_text_parser = []
for i in range(len(doc)):
    if doc[i].lemma_ in ['скидка', 'взнос', 'стоимость', 'цена', 'оплата', 'предоплата', 'ставка', 'ипотека', 'кредит']:
        # анализируемые сущности
        inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'entity'}]
    elif doc[i].lemma_ in ['процент', 'рубль', 'метр', 'квадрат', 'год', 'квартал', 'неделя', 'месяц', 'день', 'условие', 'этаж', 'корпус']:
        # измерения для чисел
        inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'measure', 'relation': -1}]
    elif doc[i].like_num:
        # числа
        inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'value', 'relation': doc[i].head.i}]

# корректируем для чисел их отношение к измерению
for i in range(len(inp_text_parser)):
    if inp_text_parser[i]['code'] == 'value':
        _ = correct_relation(inp_text_parser, inp_text_parser[i]['pos'])

# удалим токены-числа и токены-измерения, если они относятся не к измерениям "процент" или "рубль"
length_parser = len(inp_text_parser)
i = 0
while i < length_parser:
    if inp_text_parser[i]['code'] == 'value':
        j = find_pos(inp_text_parser, inp_text_parser[i]['relation'])
        if (j == -1) or not(inp_text_parser[j]['lemma'] in ['процент', 'рубль']):
            _ = inp_text_parser.pop(i)
            length_parser -= 1
        else:
            i += 1
    elif (inp_text_parser[i]['code'] == 'measure') and not(inp_text_parser[i]['lemma'] in ['процент', 'рубль']):
        _ = inp_text_parser.pop(i)
        length_parser -= 1
    else:
        i += 1

# расчитаем расстояния между сущностями и началом блоков чисел и измерения слева и справа
# если между сущностями отсутствуют числа и измерения, то установим -1
length_parser = len(inp_text_parser)
for i in range(length_parser):
    if inp_text_parser[i]['code'] == 'entity':
        left = -1
        if (i-1 >= 0) and (inp_text_parser[i-1]['code'] in ['value', 'measure']):
            left = abs(inp_text_parser[i]['pos'] - inp_text_parser[i-1]['pos'])
        inp_text_parser[i]['left'] = left
        right = -1
        if (i+1 < length_parser) and (inp_text_parser[i+1]['code'] in ['value', 'measure']):
            right = abs(inp_text_parser[i]['pos'] - inp_text_parser[i+1]['pos'])
        inp_text_parser[i]['right'] = right

# удалим сущности не "скидка", у которых растояния одновременно слева и справа -1
length_parser = len(inp_text_parser)
i = 0
while i < length_parser:
    if (inp_text_parser[i]['code'] == 'entity') and not(inp_text_parser[i]['lemma'] == 'скидка') and\
        ((inp_text_parser[i]['left'] == -1) and (inp_text_parser[i]['right'] == -1)):
        _ = inp_text_parser.pop(i)
        length_parser -= 1
    else:
        i += 1

In [ ]:
# установим для чисел и измерений их отношение к сущностям
length_parser = len(inp_text_parser)
for i in range(length_parser):
    if inp_text_parser[i]['code'] == 'entity':
        if (inp_text_parser[i]['left'] == -1) or ((inp_text_parser[i]['left'] >= inp_text_parser[i]['right']) and (inp_text_parser[i]['right'] != -1)):
            j = i+1
            print(i,j)
            while (j < length_parser) and (inp_text_parser[j]['code'] != 'entity'):
                j += 1
            print(i,j)
            fl = False
            if j == length_parser:
                j -= 1
                fl = True
            if (j < length_parser) and (inp_text_parser[j]['code'] == 'entity'):
                fl = True

            if i != j-1:
                if (j < length_parser) and ((inp_text_parser[j]['code'] == 'entity') and (inp_text_parser[i]['right'] < inp_text_parser[j]['left'])):
                    j -= 1
                if (j == length_parser) or (fl and (inp_text_parser[j+1]['code'] == 'entity') and (inp_text_parser[i]['right'] < inp_text_parser[j+1]['left'])):
                    pos = inp_text_parser[i]['pos']
                    for k in range(i+1, j):
                        inp_text_parser[k]['relation'] = pos
                        inp_text_parser[k]['marker'] = 'I-value'
                    inp_text_parser[i+1]['marker'] = 'B-value'
        elif (inp_text_parser[i]['right'] == -1) or ((inp_text_parser[i]['left'] < inp_text_parser[i]['right']) and (inp_text_parser[i]['left'] != -1)):
            j = i-1
            print(i,j)
            while (j >= 0) and (inp_text_parser[j]['code'] != 'entity'):
                j -= 1
            print(i,j)
            if (i != j+1):
                if (j >= 0) and ((inp_text_parser[j]['code'] == 'entity') and (inp_text_parser[i]['left'] < inp_text_parser[j]['right'])):
                    j += 1
                pos = inp_text_parser[i]['pos']
                for k in range(j+1, i):
                    inp_text_parser[k]['relation'] = pos
                    inp_text_parser[k]['marker'] = 'I-value'
                inp_text_parser[j+1]['marker'] = 'B-value'

0 1
0 3


IndexError: list index out of range

In [ ]:
inp_text_parser

[{'pos': 0,
  'text': 'скидку',
  'lemma': 'скидка',
  'code': 'entity',
  'left': -1,
  'right': 7},
 {'pos': 7,
  'text': 'три',
  'lemma': 'три',
  'code': 'value',
  'relation': 0,
  'marker': 'B-value'},
 {'pos': 8,
  'text': 'процента',
  'lemma': 'процент',
  'code': 'measure',
  'relation': 11,
  'marker': 'B-value'},
 {'pos': 11,
  'text': 'скидка',
  'lemma': 'скидка',
  'code': 'entity',
  'left': 3,
  'right': -1}]

In [ ]:
# удалим токены-числа и токены-измерения, если они относятся не к сущности "скидка"
length_parser = len(inp_text_parser)
i = 0
while i < length_parser:
    if inp_text_parser[i]['code'] in ['value', 'measure']:
        j = find_pos(inp_text_parser, inp_text_parser[i]['relation'])
        if (j == -1) or not(inp_text_parser[j]['lemma'] in ['скидка']):
            _ = inp_text_parser.pop(i)
            length_parser -= 1
        else:
            i += 1
    elif inp_text_parser[i]['code'] == 'entity':
        if inp_text_parser[i]['lemma'] in ['скидка']:
            inp_text_parser[i]['marker'] = 'B-discount'
            i += 1
        else:
            _ = inp_text_parser.pop(i)
            length_parser -= 1
    else:
        i += 1

In [ ]:
inp_text_parser

[{'pos': 3,
  'text': 'скидку',
  'lemma': 'скидка',
  'code': 'entity',
  'marker': 'B-discount'}]